In [1]:
import pandas as pd
import numpy as np
import re

Import Data

In [2]:
cities = pd.read_csv("../data/raw/PLACES__Census_Tract_Data__GIS_Friendly_Format___2021_release_20250804.csv", dtype={"TractFIPS": str})
fara = pd.read_csv("../data/raw/food_access/Food Access Research Atlas.csv", dtype={"CensusTract": str})
cal_acs5 = pd.read_csv("../data/raw/ACS5 2019.csv", dtype={'GEOID': str})

Get df shapes

In [3]:
print(f'cities: {cities.shape}, fara: {fara.shape}, cal_asc5: {cal_acs5.shape}')

cities: (72337, 67), fara: (72531, 147), cal_asc5: (8057, 5)


Filter for California and prep GEOID column for future merge

In [4]:
cities = cities.rename(columns={"TractFIPS": "GEOID"})
fara = fara.rename(columns={"CensusTract": "GEOID"})

calif_cities = cities[cities['StateDesc'] == 'California']

calif_food_access =  fara[fara['State'] == 'California']

calif_food_access.loc[:, "GEOID"] = "0" + calif_food_access['GEOID']

calif_cities.loc[:, "GEOID"] = calif_cities["GEOID"]

print(f'cities: {calif_cities.shape}, FARA: {calif_food_access.shape}')


cities: (8005, 67), FARA: (8024, 147)


View columns of 500 Cities dataset

In [5]:
calif_cities.columns
calif_cities.GEOID

1        06067007402
5        06037221601
6        06037402404
7        06059011102
8        06037294200
            ...     
17656    06111008001
17658    06075015300
17659    06081605400
17660    06085509302
17665    06075010100
Name: GEOID, Length: 8005, dtype: object

View columns fo FARA dataset

In [6]:
print(calif_food_access.columns.to_list())

['GEOID', 'State', 'County', 'Urban', 'Pop2010', 'OHU2010', 'GroupQuartersFlag', 'NUMGQTRS', 'PCTGQTRS', 'LILATracts_1And10', 'LILATracts_halfAnd10', 'LILATracts_1And20', 'LILATracts_Vehicle', 'HUNVFlag', 'LowIncomeTracts', 'PovertyRate', 'MedianFamilyIncome', 'LA1and10', 'LAhalfand10', 'LA1and20', 'LATracts_half', 'LATracts1', 'LATracts10', 'LATracts20', 'LATractsVehicle_20', 'LAPOP1_10', 'LAPOP05_10', 'LAPOP1_20', 'LALOWI1_10', 'LALOWI05_10', 'LALOWI1_20', 'lapophalf', 'lapophalfshare', 'lalowihalf', 'lalowihalfshare', 'lakidshalf', 'lakidshalfshare', 'laseniorshalf', 'laseniorshalfshare', 'lawhitehalf', 'lawhitehalfshare', 'lablackhalf', 'lablackhalfshare', 'laasianhalf', 'laasianhalfshare', 'lanhopihalf', 'lanhopihalfshare', 'laaianhalf', 'laaianhalfshare', 'laomultirhalf', 'laomultirhalfshare', 'lahisphalf', 'lahisphalfshare', 'lahunvhalf', 'lahunvhalfshare', 'lasnaphalf', 'lasnaphalfshare', 'lapop1', 'lapop1share', 'lalowi1', 'lalowi1share', 'lakids1', 'lakids1share', 'laseniors1

Drop irrelevant columns

In [7]:
drop_cols = [
    "lapop1share",
    "laasianhalfshare",
    "lapophalf",
    "lapophalfshare",
    "lalowihalf",
    "lalowihalfshare",
    "lakidshalf",
    "lakidshalfshare",
    "laseniorshalf",
    "laseniorshalfshare",
    "lawhitehalf",
    "lawhitehalfshare",
    "lablackhalf",
    "lablackhalfshare",
    "laasianhalf",
    "laaianhalf",
    "lanhopihalf",
    "lanhopihalfshare",
    "lasnaphalfshare",
    "laaianhalfshare",
    "laomultirhalf",
    "laomultirhalfshare",
    "lahisphalf",
    "lahisphalfshare",
    "lahunvhalf",
    "lahunvhalfshare",
    "lasnaphalf",
    "lapop1", 
    "TractSNAP",
    "TractHUNV",
    "TractHispanic",
    "TractOMultir",
    "TractAIAN",
    "TractNHOPI",
    "TractAsian",
    "TractBlack",
    "TractWhite",
    "TractSeniors",
    "TractKids",
    "lasnap20share",
    "lasnap20",
    "lahunv20share",
    "lahisp20share",
    "lahisp20",
    "laomultir20share",
    "laomultir20",
    "laaian20share",
    "laaian20",
    "lanhopi20share",
    "lanhopi20",
    "laasian20share",
    "laasian20",
    "lablack20share",
    "lablack20",
    "lawhite20share",
    "lawhite20",
    "laseniors20share",
    "laseniors20",
    "lakids20share",
    "lakids20",
    "lalowi20share",
    "lalowi20",
    "lapop20share",
    "lapop20",
    "lasnap10share",
    "lasnap10",
    "lahunv10share",
    "lahunv10",
    "lahisp10share",
    "lahisp10",
    "laomultir10share",
    "laomultir10",
    "laaian10share",
    "laaian10",
    "lanhopi10share", 
    "lanhopi10",
    "laasian10share",
    "laasian10",
    "lablack10share",
    "lablack10",
    "lawhite10share",
    "lawhite10",
    "laseniors10share",
    "laseniors10",
    "lakids10share",
    "lakids10",
    "lalowi10share",
    "lalowi10",
    "lapop10share",
    "lapop10",
    "lasnap1share",
    "lasnap1",
    "lahunv1share",
    "lahunv1",
    "lahisp1share",
    "lahisp1", 
    "laomultir1share",
    "laomultir1",
    "laaian1share",
    "laaian1",
    "lanhopi1share",
    "lanhopi1",
    "laasian1share",
    "laasian1",
    "lablack1share",
    "lablack1",
    "lawhite1share",
    "lawhite1",
    "laseniors1share",
    "laseniors1",
    "lakids1share",
    "lakids1",
    "lalowi1share",
    "lalowi1",
    "GroupQuartersFlag",
    "lahunv20",
    "NUMGQTRS",
    "PCTGQTRS",
    "State"
]

In [8]:
calif_food_access = calif_food_access.drop(columns=drop_cols)

Treat calif_food_access columns

In [9]:
calif_food_access.apply(pd.unique).to_frame(name="Unique Values")

,Unique Values
GEOID,"[06001400100, 06001400200, 06001400300, 060014..."
County,"[Alameda County, Alpine County, Amador County,..."
Urban,"[1, 0]"
Pop2010,"[2937, 1974, 4865, 3703, 3517, 1571, 4206, 359..."
OHU2010,"[1314, 856, 2418, 1779, 1578, 690, 1755, 1567,..."
LILATracts_1And10,"[0, 1]"
LILATracts_halfAnd10,"[0, 1]"
LILATracts_1And20,"[0, 1]"
LILATracts_Vehicle,"[0, 1]"
HUNVFlag,"[0, 1]"


In [10]:
calif_food_access.loc[:, "LAPOP1_10"] = calif_food_access["LAPOP1_10"].fillna(0)
calif_food_access.loc[:, "LAPOP05_10"] = calif_food_access["LAPOP05_10"].fillna(0)
calif_food_access.loc[:, "LAPOP1_20"] = calif_food_access["LAPOP1_20"].fillna(0)
calif_food_access.loc[:, "LALOWI1_10"] = calif_food_access["LALOWI1_10"].fillna(0)
calif_food_access.loc[:, "LALOWI05_10"] = calif_food_access["LALOWI05_10"].fillna(0)
calif_food_access.loc[:, "LALOWI1_20"] = calif_food_access["LALOWI1_20"].fillna(0)
calif_food_access.apply(pd.unique).to_frame(name="Unique Values")

,Unique Values
GEOID,"[06001400100, 06001400200, 06001400300, 060014..."
County,"[Alameda County, Alpine County, Amador County,..."
Urban,"[1, 0]"
Pop2010,"[2937, 1974, 4865, 3703, 3517, 1571, 4206, 359..."
OHU2010,"[1314, 856, 2418, 1779, 1578, 690, 1755, 1567,..."
LILATracts_1And10,"[0, 1]"
LILATracts_halfAnd10,"[0, 1]"
LILATracts_1And20,"[0, 1]"
LILATracts_Vehicle,"[0, 1]"
HUNVFlag,"[0, 1]"


Treat cities dataframe

In [11]:
calif_cities.apply(pd.unique).to_frame(name="Unique Values")

,Unique Values
StateAbbr,[CA]
StateDesc,[California]
CountyName,"[Sacramento, Los Angeles, Orange, Riverside, N..."
CountyFIPS,"[6067, 6037, 6059, 6065, 6055, 6001, 6015, 601..."
GEOID,"[06067007402, 06037221601, 06037402404, 060590..."
...,...
STROKE_CrudePrev,"[3.9, 2.9, 0.3, 2.7, 2.6, 5.6, 4.2, 4.0, 2.1, ..."
STROKE_Crude95CI,"[( 3.6, 4.3), ( 2.6, 3.3), ( 0.3, 0.4), ( 2..."
TEETHLOST_CrudePrev,"[18.2, 14.8, 8.1, 12.3, 12.1, 19.7, 23.8, 18.8..."
TEETHLOST_Crude95CI,"[(13.7, 23.5), (10.7, 19.6), ( 5.5, 12.1), ( 9..."


In [12]:
calif_cities = calif_cities.drop(columns=["StateAbbr", "StateDesc", "CountyFIPS"])
calif_cities.columns

Index(['CountyName', 'GEOID', 'TotalPopulation', 'ACCESS2_CrudePrev',
       'ACCESS2_Crude95CI', 'ARTHRITIS_CrudePrev', 'ARTHRITIS_Crude95CI',
       'BINGE_CrudePrev', 'BINGE_Crude95CI', 'BPHIGH_CrudePrev',
       'BPHIGH_Crude95CI', 'BPMED_CrudePrev', 'BPMED_Crude95CI',
       'CANCER_CrudePrev', 'CANCER_Crude95CI', 'CASTHMA_CrudePrev',
       'CASTHMA_Crude95CI', 'CERVICAL_CrudePrev', 'CERVICAL_Crude95CI',
       'CHD_CrudePrev', 'CHD_Crude95CI', 'CHECKUP_CrudePrev',
       'CHECKUP_Crude95CI', 'CHOLSCREEN_CrudePrev', 'CHOLSCREEN_Crude95CI',
       'COLON_SCREEN_CrudePrev', 'COLON_SCREEN_Crude95CI', 'COPD_CrudePrev',
       'COPD_Crude95CI', 'COREM_CrudePrev', 'COREM_Crude95CI',
       'COREW_CrudePrev', 'COREW_Crude95CI', 'CSMOKING_CrudePrev',
       'CSMOKING_Crude95CI', 'DENTAL_CrudePrev', 'DENTAL_Crude95CI',
       'DEPRESSION_CrudePrev', 'DEPRESSION_Crude95CI', 'DIABETES_CrudePrev',
       'DIABETES_Crude95CI', 'GHLTH_CrudePrev', 'GHLTH_Crude95CI',
       'HIGHCHOL_CrudePrev',

Merge Data on GEOID column

In [13]:
merged = pd.merge(calif_cities, calif_food_access, on='GEOID', how='inner')
merged = pd.merge(merged, cal_acs5, on="GEOID", how="inner")

merged.shape

(8004, 95)

Export

In [14]:
merged.to_csv("../data/processed/Fara+500+.csv")